In [ ]:
!pip install torch
!pip install numpy
!pip install tiktoken


In [ ]:
import pandas as pd
import re
import torch
import tiktoken

import util
import bigram

In [ ]:
with open('Conversation.csv', 'r', encoding = 'utf-8') as f: 
    text= f.read()

print(f"length of dataset in characters: {len(text)}")

print(text[:1000])

Filtering Dataset

In [ ]:
question_answer_text = re.split(r'(?<! ),(?! )' , text )

lines = text.split('\n')

text = ""
for line in lines:
    parts = line.split(',')
    if len(parts) >= 3:  # Ensure at least 3 parts exist before accessing indices
        question = parts[1].strip('"')
        answer = parts[2].strip('"')
        text += f"Question: {question}\nAnswer: {answer}\n\n"

print(text)

chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

In [ ]:
char_to_int = {c: i for i , c in enumerate(chars)}
int_to_char = {i: c for i , c in enumerate(chars)}

decode_text = lambda l: ''.join([int_to_char[i] for i in l]) # decodes a list of integers as a string.  
encode_text = lambda str: [char_to_int[c] for c in str] # encodes a string into a list of integers

print(encode_text("hi there"))
print(decode_text(encode_text("hi there")))

In [ ]:
data = torch.tensor(encode_text(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

In [ ]:
data = torch.tensor(encode_text(text), dtype=torch.long)

#split data into training, validation, and test sets
train= int(0.8 * len(data))
val = int(0.9 * len(data))
train_data = data[:train]
val_data = data[train:val]
test_data = data[val:]

# print(train_data)

In [ ]:
block_size = 8
train_data[:block_size]

In [ ]:
import utili
torch.manual_seed(utili.MANUAL_SEED)

batch_size = 4
block_size = 8

def get_batch(split):
  if split == 'train':
    data = train_data
  elif split == 'val':
    data = val_data
  else:
    data = test_data  

  idx = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in idx])
  y = torch.stack([data[i+1:i+block_size+1] for i in idx])
  return x, y

xb, yb = get_batch('train')

print('context:', xb.shape, xb)
print('target:', yb.shape, yb)

for b in range(batch_size): 
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"context: {context.tolist()}, target: {target}")